In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from FAA import Ohare
from FAA import Hanger
import pandas as pd
from sqlalchemy import create_engine


In [2]:
faa = Ohare.Ohare()

In [4]:
faa.ord_flights.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2018,1,1,27,6,2018-01-27,UA,19977,UA,N16217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,1,1,27,6,2018-01-27,UA,19977,UA,N811UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,1,27,6,2018-01-27,UA,19977,UA,N54711,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,1,27,6,2018-01-27,UA,19977,UA,N771UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,1,27,6,2018-01-27,UA,19977,UA,N835UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
faa.ord_flights.groupby(faa.ord_flights['DayOfWeek']).count()

,Year,Quarter,Month,DayofMonth,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
DayOfWeek,,,,,,,,,,,,,,,,,,,,,
1,8586,8586,8586,8586,8586,8586,8586,8586,8577,8586,...,0,0,0,0,0,0,0,0,0,0
2,8416,8416,8416,8416,8416,8416,8416,8416,8410,8416,...,0,0,0,0,0,0,0,0,0,0
3,8509,8509,8509,8509,8509,8509,8509,8509,8496,8509,...,0,0,0,0,0,0,0,0,0,0
4,6953,6953,6953,6953,6953,6953,6953,6953,6888,6953,...,0,0,0,0,0,0,0,0,0,0
5,6996,6996,6996,6996,6996,6996,6996,6996,6978,6996,...,0,0,0,0,0,0,0,0,0,0
6,5393,5393,5393,5393,5393,5393,5393,5393,5392,5393,...,0,0,0,0,0,0,0,0,0,0
7,6645,6645,6645,6645,6645,6645,6645,6645,6641,6645,...,0,0,0,0,0,0,0,0,0,0


In [62]:
def flights_per_day(flight_bts_df):
    fpd = flight_bts_df.groupby(flight_bts_df['DayOfWeek']).count()[['Year']]
    days = pd.DataFrame(['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'], index=[1,2,3,4,5,6,7])
    days_flights = pd.concat([days, fpd], axis=1)
    return days_flights.rename(columns={'Year': 'Flights', 0: 'Day'})
    

In [63]:
flights_per_day(faa.ord_flights)

,Day,Flights
DayOfWeek,,
1,MON,8586
2,TUE,8416
3,WED,8509
4,THU,6953
5,FRI,6996
6,SAT,5393
7,SUN,6645


In [70]:
faa.ord_flights.groupby(faa.ord_flights['DayOfWeek']).sum()[['DepDel15']]

,DepDel15
DayOfWeek,
1,2631.0
2,1725.0
3,1613.0
4,932.0
5,1257.0
6,706.0
7,1370.0


In [72]:
pd.concat([flights_per_day(faa.ord_flights), faa.ord_flights.groupby(faa.ord_flights['DayOfWeek']).sum()[['DepDel15']].DepDel15], axis=1)

,Day,Flights,DepDel15
DayOfWeek,,,
1,MON,8586,2631.0
2,TUE,8416,1725.0
3,WED,8509,1613.0
4,THU,6953,932.0
5,FRI,6996,1257.0
6,SAT,5393,706.0
7,SUN,6645,1370.0


In [79]:
def flights_per_day(flight_bts_df):
    fpd = flight_bts_df.groupby(flight_bts_df['DayOfWeek']).count()[['Year']]
    days = pd.DataFrame(['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN'], index=[1,2,3,4,5,6,7])
    days_flights = pd.concat([days, fpd], axis=1)
    return days_flights.rename(columns={'Year': 'Flights', 0: 'Day'})
    

def daily_delays(flight_bts_df):
    
    fly_by_day = flights_per_day(flight_bts_df)
    
    delays = flight_bts_df.groupby(flight_bts_df['DayOfWeek']).sum()[['DepDel15']].DepDel15
    
    return pd.concat([fly_by_day, delays], axis=1)

In [78]:
daily_delays(faa.ord_flights)

,Day,Flights,DepDel15
DayOfWeek,,,
1,MON,8586,2631.0
2,TUE,8416,1725.0
3,WED,8509,1613.0
4,THU,6953,932.0
5,FRI,6996,1257.0
6,SAT,5393,706.0
7,SUN,6645,1370.0
